# Ingestão Bronze - Cadastro de Cliente

Este notebook realiza a ingestão de um arquivo Excel contendo o cadastro de clientes
e grava os dados em formato Delta na camada **Bronze**.

**Origem dos dados:**
- Arquivo: `D:\Projetos\Jornada_financas_pessoais\data\source\cadastro_cliente.xlsx`

**Destino:**
- Caminho Delta: `D:\Projetos\Jornada_financas_pessoais\data\delta\bronze\`
- Tabela Delta: `bronze.raw_cadcliente`


## Imports

In [1]:
from spark_config import init_spark
import pandas as pd
import os

## Start Spark Session

In [2]:
spark = init_spark("Leitura de arquivo de formato excel")


✅ Spark 3.5.7 iniciado com Hive local persistente!
📁 Warehouse: D:/Projetos/DataLake/spark-warehouse
📁 Metastore: D:/Projetos/DataLake/metastore_db



## Variables

In [3]:
# Define o caminho do diretório dos arquivos de origem
source_path = "D:/Projetos/Jornada_financas_pessoais/data/source"

source_file_path = f'{source_path}/Cadastro_Cliente.xlsx'

source_file_name = os.path.basename(source_file_path)

# Define caminhos locais onde serão armazenadas as tabelas Delta
base_bronze_path = "D:/Projetos/Jornada_financas_pessoais/data/delta/bronze"

# Define o caminho da tabela Delta Source
delta_path_cadcliente = f"{base_bronze_path}/raw_cadcliente"

## Read Excel File

In [4]:
# Lê o Excel com pandas
df_cadcliente_pd = pd.read_excel(
    source_file_path,
    engine='openpyxl',
    dtype=str  # Força todas as colunas como string
)

# Adiciona o nome do arquivo como nova coluna
df_cadcliente_pd['nome_arquivo'] = source_file_name

# Renomeia a coluna DATA NASCIMENTO
df_cadcliente_pd.rename(columns={'DATA NASCIMENTO': 'data_nascimento'}, inplace=True)

# Converte para PySpark DataFrame
df_cadcliente = spark.createDataFrame(df_cadcliente_pd)

print(df_cadcliente.count(), "registros lidos")

3 registros lidos


## Save Data Frame

In [5]:
# Grava em formato Delta
(
    df_cadcliente.write.format("delta")
    .mode("overwrite")
    .option("overwriteSchema", "false")
    .save(delta_path_cadcliente)
)

print(f"[SUCESSO] DataFrame gravado no Delta Lake em: {delta_path_cadcliente}")

[SUCESSO] DataFrame gravado no Delta Lake em: D:/Projetos/Jornada_financas_pessoais/data/delta/bronze/raw_cadcliente


## Stop Spark Session

In [6]:
# Encerra a SparkSession
spark.stop()